My intentions here is to cut the larger video into smaller segments based on the interval determined by timestamps from chapters

In [47]:
from pytube import YouTube
import moviepy.editor as mpe
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

import requests
import re
import os
import time
from datetime import date, datetime
import random


In [48]:
def chapther_grabber(video_link):
    #record start time
    start_time = time.time()

    yt = YouTube(video_link)
    print("video found")

    #Saved the description in a string variable. want to remove the chapterstamps out into its own list
    url = requests.get(video_link).text
    search = re.search(r'shortDescription":"', url)
    description = ""
    print("description found")

    count = search.start() + 19  # adding the length of the 'shortDescription":"
    while True:
        # get the letter at current index in text
        letter = url[count]
        if letter == "\"":
            if url[count - 1] == "\\":
                # this is case where the letter before is a backslash, meaning it is not real end of description
                description += letter
                count += 1
            else:
                break
        else:
            description += letter
            count += 1

    print(f'description: {description}')

    #Length of video - converted to timestamp format
    video_len = yt.length

    seconds = video_len % (24 * 3600)
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    video_len = ("%02d:%02d" % (minutes, seconds)) # Format --> MM:SS

    print(f"length of video {video_len} min")

    #extract timestamps
    timestamps = re.findall(r"\d+:\d+",description)
    print(f"Found {len(timestamps)} timestamps")

    #record end time
    end_time = time.time()

    print(f"The time of execution of above program is : {round((end_time-start_time),2)} s")
    
    return timestamps , video_len

In [49]:
#the only global variable is the link
ytv_url = "https://www.youtube.com/watch?v=phOkyQo7rjo"

timestamps, video_len = chapther_grabber(ytv_url)



video found
description found
description: Take a moment to escape into Spyro the Dragon's enchanting world, where gentle ambient space music wraps around you like a warm blanket. As Spyro soars through autumn skies, the blend of calming sounds and nostalgic visuals brings a sense of peace and comfort, almost like reconnecting with an old friend. It's more than just a game—it's a soothing retreat that invites you to relax, unwind, and lose yourself in the beauty of a beloved classic. CozyCorner is your place to slow down and savor the simple joys.\n\n\nSubscribe and leave a like!!\nShare your feelings in the comments below!\n\nTimestamps⏰\n00:00 \"first light of autumn\"\n04:07 \"harvest breeze\"\n07:07 \"first night of autumn\"\n11:07 \"golden leaves\"\n15:07 \"quaint village\"\n18:12 \"village market stroll\"\n20:20 \"candlelit evening\"\n21:18 \"harvest moon rising\"\n24:30 \"when will the snow fall?\"\n\n#relaxingmusic #darkambience #sleepaid #gamingnostalgia\n#ambientmusic #backgr

## From here the editing will begin!

I now have the timestamps in a list, the full video length, and the downloaded video

From these variables i should be able to determine where moviepy.editor should cut the video

His system works differently. it cuts every 30 seconds. I want to cut by intervals.

so whats the math?
- convert timestamps to seconds so i can determine intervals
- add desired length of clip from start of timestamp which is then the end.
- Add noise to see if different length of clips improve quality

In [4]:
def time_to_seconds(timestamps):
    minutes, seconds = map(int, timestamps.split(":"))
    return minutes * 60 + seconds

In [50]:
def shortMaker(clip_length,video_path, video_link):

    ytv_url = "https://www.youtube.com/watch?v=phOkyQo7rjo"
    timestamps, video_len = chapther_grabber(ytv_url)

    def time_to_seconds(timestamps):
        minutes, seconds = map(int, timestamps.split(":"))
        return minutes * 60 + seconds

    #record start time
    start = time.time()

    #i want to add this to the output path
    current_date = date.today().strftime("%d-%m-%y")

    #convert timestamps to seconds
    timestamps_in_seconds = [time_to_seconds(t) for t in timestamps]

    #create output path for shorts videos
    SAVE_PATH = f"/home/mali/repurpose/shortsfolder/{current_date}" 
    if not os.path.exists(SAVE_PATH):
        os.makedirs(SAVE_PATH)

    #try:
    for i in range(len(timestamps)):
        print("Processing subclip")
        noise = 0
        #determine start and end of clips
        noise = random.randint(0, min(40,clip_length))
        start_time = timestamps_in_seconds[i]
        end_time = (timestamps_in_seconds[i] + clip_length)  - noise

        print(f"clip start:{start_time} clip end:{end_time} clip length:{end_time-start_time}")

        ffmpeg_extract_subclip(
                    video_path,
                    start_time,
                    end_time,
                    targetname=f"{SAVE_PATH}/short{i+1}.mp4")
            
        print(f"clip{i} finished...")
    #except Exception as e:
     #   print(f"Error: {e}")
    
    #record end time
    end = time.time()
    print(f"The time of execution of above program is : {round((end-start),2)} s")

In [51]:
shortMaker(55,"/home/mali/repurpose/videofolder/Escape Into Peaceful Autumn Nostalgia | Fall Ambient Music - Rec.5_concat.mp4")

Processing subclip
clip start:0 clip end:53 clip length:53
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip0 finished...
Processing subclip
clip start:247 clip end:295 clip length:48
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip1 finished...
Processing subclip
clip start:427 clip end:472 clip length:45
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip2 finished...
Processing subclip
clip start:667 clip end:698 clip length:31
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip3 finished...
Processing subclip
clip start:907 clip end:962 clip length:55
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip4 finished...
Processing subclip
clip start:1092 clip end:1140 clip length:48
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip5 finished...
Processing subclip
clip start:1220 clip end:1265 clip length:45
Moviepy - Running:
>>> "+ " ".join(cmd)
Mov

What i might add


- customizable pathing (currently you have to write it in)
- make noise optional by creating variables for range of noise